<a href="https://colab.research.google.com/github/Driante/fantacalcio/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import io, os
from collections import deque 
from src.calendario import *
from itertools import permutations
import matplotlib.pyplot as plt

In [2]:
calendar_raw_name = 'Calendario_Campionato---A-buon-rendere.xlsx'
calendar_raw = import_calendar(calendar_raw_name)
df_persquadra,ordine_giornate = clean_calendar(calendar_raw)
squadre =df_persquadra.index.values
df_persquadra

,Fantapunti,Avversari,Gol
Gargantua,"[64.0, 66.5, 77.5, 70.5, 74.0, 60.0, 71.0, 79....","[Capitan Martella, CACCOLA F.C., Barriera-team...","[0.0, 1.0, 3.0, 2.0, 3.0, 0.0, 2.0, 4.0, 0.0, ..."
asdMalerba,"[74.0, 86.0, 66.0, 62.5, 78.5, 65.0, 67.0, 67....","[CACCOLA F.C., Paris FC, SamPDORRR FC, Capitan...","[3.0, 6.0, 1.0, 0.0, 4.0, 0.0, 1.0, 1.0, 0.0, ..."
Civediamoamaggio,"[70.5, 75.5, 78.5, 63.5, 67.0, 63.5, 69.0, 72....","[Paris FC, SamPDORRR FC, Capitan Martella, Gar...","[2.0, 3.0, 4.0, 0.0, 1.0, 0.0, 1.0, 2.0, 1.0, ..."
Barriera-team,"[70.0, 75.0, 70.5, 69.0, 69.5, 78.0, 70.5, 79....","[SamPDORRR FC, Capitan Martella, Gargantua , C...","[2.0, 3.0, 2.0, 1.0, 1.0, 4.0, 2.0, 4.0, 2.0, ..."
Capitan Martella,"[83.0, 74.5, 62.5, 78.0, 72.0, 70.5, 76.5, 77....","[Gargantua , Barriera-team, Civediamoamaggio ,...","[5.0, 3.0, 0.0, 4.0, 2.0, 2.0, 3.0, 3.0, 2.0, ..."
CACCOLA F.C.,"[79.0, 72.0, 77.5, 72.0, 74.0, 75.5, 70.0, 72....","[asdMalerba, Gargantua , Paris FC, Barriera-te...","[4.0, 2.0, 3.0, 2.0, 3.0, 3.0, 2.0, 2.0, 3.0, ..."
Paris FC,"[70.5, 78.5, 70.5, 70.0, 73.0, 73.5, 71.0, 87....","[Civediamoamaggio , asdMalerba, CACCOLA F.C., ...","[2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 6.0, 2.0, ..."
SamPDORRR FC,"[83.5, 82.5, 73.5, 70.5, 73.5, 65.5, 62.0, 70....","[Barriera-team, Civediamoamaggio , asdMalerba,...","[5.0, 5.0, 2.0, 2.0, 2.0, 0.0, 0.0, 2.0, 1.0, ..."


In [3]:
def crea_classifica(df_persquadra):
    ngiornate = (df_persquadra.loc[df_persquadra.index[0]]['Fantapunti'] ==0).argmax()
    nsquadre = df_persquadra.shape[0]
    df_persquadra['Punti'] = [np.zeros(ngiornate) for _ in range(nsquadre)]
    for giornata in range(ngiornate):
        for squadra in range(nsquadre):
            g1 = df_persquadra['Gol'][squadra][giornata]
            g2 = df_persquadra.loc[df_persquadra['Avversari'][squadra][giornata]]['Gol'][giornata]
            if g1> g2:
                df_persquadra['Punti'][squadra][giornata] += 3
            elif g1==g2:
                df_persquadra['Punti'][squadra][giornata] += 1
    return 

def crea_calendario_template(nsquadre):
    #Crea un template di calendario di girone all'italiana
    squadre_template = ['A','B','C','D','E','F','G','H']
    calendario_template = []
    lista_avversari=[[] for _ in range(nsquadre)]
    for shift in range(nsquadre-1):
        temp = deque(squadre_template[1:])
        temp.rotate(shift)
        squadre_template_rot = [squadre_template[0]] + list(temp)
        giornata = []

        for n in range(int(nsquadre/2)):
            giornata.append(squadre_template_rot[n] + '-' + squadre_template_rot[-n-1])
            lista_avversari[squadre_template.index(squadre_template_rot[n])].append(squadre_template_rot[-n-1])
            lista_avversari[squadre_template.index(squadre_template_rot[-n-1])].append(squadre_template_rot[n])
        calendario_template.append(giornata)
    return squadre_template,calendario_template,lista_avversari

def tutti_calendari(squadre):
    #permuta tutti i possibli assegnamenti delle squadre al template, generando tutti i possibili calendari 
    #sotto forma di liste di avversari
    nsquadre = len(squadre)
    squadre_template,calendario_template,lista_avversari = crea_calendario_template(nsquadre)
    cDict_template = {squadre_template[n] : lista_avversari[n] for n in range(nsquadre)}
    all_calendars = []
    i= 0
    for squadre_perm in permutations(squadre):
        cDictnew = {}
        template_matching = {squadre_template[n] : squadre_perm[n] for n in range(nsquadre)}
        for n in range(nsquadre):
            cDictnew[squadre_perm[n]] = [template_matching.get(a,'default') for a in cDict_template[squadre_template[n]]]*7
        all_calendars.append(cDictnew)
        i+=1
    return all_calendars

In [4]:
all_calendars = tutti_calendari(squadre)

In [5]:
df_persquadra_mut = df_persquadra.copy()

In [6]:
classifiche = [];
ncal = 0
for cal in all_calendars:
    for s in cal:
        df_persquadra_mut.loc[s]['Avversari'] = cal[s]
    crea_classifica(df_persquadra_mut)
    classifiche.append(df_persquadra_mut['Punti'].apply(np.sum).sort_values(ascending=False))
    #print(ncal, "Capolista: ",classifiche[ncal].index[0])
    ncal +=1

In [7]:
pv = {s : 0 for s in squadre}
FPunti = {s : [] for s in squadre}
MediaPunti = {s : 0 for s in squadre}
cc = []
i=0
for classifica in classifiche:
    pv[classifica.index[0]] += (1/np.math.factorial(8))*100
    for idx in classifica.index:
        MediaPunti[idx] += classifica[idx]/ np.math.factorial(8)
        FPunti[idx].append(classifica[idx])
    if (classifica[0] - classifica[1])>11:
        cc.append(classifica)

In [8]:
res = dict() 
for key in pv: 
    res[key] = round(pv[key], 2) 
    
print(str(res))  

{'Gargantua ': 1.04, 'asdMalerba': 0.61, 'Civediamoamaggio ': 29.96, 'Barriera-team': 12.33, 'Capitan Martella': 44.12, 'CACCOLA F.C.': 7.82, 'Paris FC': 3.61, 'SamPDORRR FC': 0.51}


In [9]:
MediaPunti

{'Gargantua ': 26.285714285714732,
 'asdMalerba': 25.571428571428665,
 'Civediamoamaggio ': 34.000000000003325,
 'Barriera-team': 32.42857142857096,
 'Capitan Martella': 35.71428571428885,
 'CACCOLA F.C.': 29.142857142857586,
 'Paris FC': 28.42857142857124,
 'SamPDORRR FC': 25.28571428571389}

---

In [10]:
len(cc)/np.math.factorial(8)*100

1.9196428571428572

In [11]:
np.where(np.array([c.index[0] == 'Civediamoamaggio ' for c in cc]))

(array([  2,  81,  85, 151, 155, 269, 270, 271, 272, 273, 277, 279, 283,
        292, 293, 294, 309, 317, 319, 326, 434, 450, 451, 487, 488, 490,
        492, 568, 612, 636, 638, 639, 641, 643, 644, 646, 651, 652, 700,
        712, 716, 717, 718, 719, 720, 721, 722, 723, 743], dtype=int64),)

In [12]:
cc[4]

Capitan Martella     47.0
Civediamoamaggio     32.0
Paris FC             32.0
Gargantua            31.0
Barriera-team        31.0
asdMalerba           29.0
SamPDORRR FC         23.0
CACCOLA F.C.         17.0
Name: Punti, dtype: float64

In [13]:
[(s,np.var(FPunti[s])) for s in squadre]

[('Gargantua ', 20.25170068027211),
 ('asdMalerba', 15.530612244897958),
 ('Civediamoamaggio ', 15.095238095238095),
 ('Barriera-team', 11.197278911564627),
 ('Capitan Martella', 23.775510204081634),
 ('CACCOLA F.C.', 28.265306122448976),
 ('Paris FC', 21.768707482993197),
 ('SamPDORRR FC', 16.48979591836735)]